## Import Libraries

In [31]:
import pandas as pd
import numpy as np
import time
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
#from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import neighbors
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

## Import Project Datasets (Apache)


In [32]:
#All projects are imported here one by one
#variables are adjusted by the sprint progress (30%,50%,80%)
apache_issue_30 = pd.read_csv("Apache/apache_issue_80.csv") #sprint issues at xx% progress
apache_iter_30 = pd.read_csv("Apache/apache_iteration_80.csv") #sprint at xx% progress

In [33]:
#check data dimension of iteration
apache_iter_30.shape

(348, 19)

In [34]:
#check data dimension of issues
apache_issue_30.shape

(5826, 14)

## Data Preprocessing

### Drop unneccesary colums


In [35]:
#columns below are irrelevant with the available dataset
apache_issue_30.drop(['no_issuelink', 'gunning_fog'], axis=1, inplace=True)
apache_issue_30.head()

boardid  sprintid         type priority  no_comment  no_affectversion  \
0        1         8         Task    Minor           1                 0   
1        1         8  Improvement    Major           1                 0   
2        1         8  Improvement    Major           3                 0   
3        1        12          Bug    Major           2                 1   
4        1        12          Bug    Minor           2                 0   

   no_fixversion  no_blocking  no_blockedby  no_fixversion_change  \
0              0            0             0                     1   
1              0            0             0                     1   
2              0            0             0                     1   
3              0            0             0                     0   
4              0            0             0                     1   

   no_priority_change  no_des_change  
0                   0              0  
1                   0              0  
2                   0              0  
3                   0              0  
4                   0              0

In [36]:
#X = apache_iter_30['vel_diff','planday','no_issue_starttime','vel_starttime','no_issue_added',
#'vel_added','no_issue_removed','vel_removed','no_issuetodo','vel_todo','no_issueinprogress',
#'vel_inprogress','no_issuedone','vel_done','no_admin','no_teammember']
#Y = apache_iter_30['vel_diff','planday','no_issue_starttime','vel_starttime','no_issue_added',
#'vel_added','no_issue_removed','vel_removed','no_issuetodo','vel_todo','no_issueinprogress',
#'vel_inprogress','no_issuedone','vel_done','no_admin','no_teammember']
#col_drop = ['boardid','sprintid','name','vel_diff']
#X = apache_iter_30.drop(col_drop,axis=1)
#Y = apache_iter_30['vel_diff']

#X = pd.get_dummies(X, prefix_sep='_')
#Y = LabelEncoder().fit_transform(Y)
#X = StandardScaler().fit_transform(X)

In [37]:
# def forest_test(X, Y):
#     X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, 
#                                                         test_size = 0.30, 
#                                                         random_state = 10)
#     start = time.process_time()
#     trainedforest = RandomForestClassifier(n_estimators=300).fit(X_Train,Y_Train)
#     print(time.process_time() - start)
#     predictionforest = trainedforest.predict(X_Test)
#     accuracy(Y_Test,predictionforest)

In [38]:
#forest_test(X, Y)

### Altering issue dataset for building the prediction model

#### Encode "priority" Column

In [39]:
#change priority column, convert from categorical data to numerical data
cleanup_num = {"priority" : {"Trivial":1,"Minor":2,"Major":3,"Critical":4,"Blocker":5}
}
apache_issue_30 = apache_issue_30.replace(cleanup_num)



#### Encode "type" Column

In [40]:
#creating columns with binary value related to issue type
issues_temp = pd.get_dummies(apache_issue_30[["type"]])


#### Merge Issue Dataset

In [41]:
apache_issue_30_merge = pd.concat([apache_issue_30,issues_temp],axis=1)
#drop original "type" column
apache_issue_30_merge.drop("type",axis=1,inplace=True)
apache_issue_30_merge.head()

boardid  sprintid  priority  no_comment  no_affectversion  no_fixversion  \
0        1         8         2           1                 0              0   
1        1         8         3           1                 0              0   
2        1         8         3           3                 0              0   
3        1        12         3           2                 1              0   
4        1        12         2           2                 0              0   

   no_blocking  no_blockedby  no_fixversion_change  no_priority_change  ...  \
0            0             0                     1                   0  ...   
1            0             0                     1                   0  ...   
2            0             0                     1                   0  ...   
3            0             0                     0                   0  ...   
4            0             0                     1                   0  ...   

   type_Epic  type_Improvement  type_New Feature  type_Story  type_Sub-task  \
0          0                 0                 0           0              0   
1          0                 1                 0           0              0   
2          0                 1                 0           0              0   
3          0                 0                 0           0              0   
4          0                 0                 0           0              0   

   type_Task  type_Technical task  type_Test  type_Umbrella  type_Wish  
0          1                    0          0              0          0  
1          0                    0          0              0          0  
2          0                    0          0              0          0  
3          0                    0          0              0          0  
4          0                    0          0              0          0  

[5 rows x 23 columns]

In [42]:
apache_issue_30_merge.columns

Index(['boardid', 'sprintid', 'priority', 'no_comment', 'no_affectversion',
       'no_fixversion', 'no_blocking', 'no_blockedby', 'no_fixversion_change',
       'no_priority_change', 'no_des_change', 'type_Bug', 'type_Documentation',
       'type_Epic', 'type_Improvement', 'type_New Feature', 'type_Story',
       'type_Sub-task', 'type_Task', 'type_Technical task', 'type_Test',
       'type_Umbrella', 'type_Wish'],
      dtype='object')

#### Aggregate issue dataset by "boardid" & "sprintid" columns

In [43]:
aggr_apache_issue = apache_issue_30_merge.groupby(['boardid','sprintid']).mean()
aggr_apache_issue

priority  no_comment  no_affectversion  no_fixversion  \
boardid sprintid                                                          
1       8         2.666667    1.666667          0.000000       0.000000   
        12        2.500000    2.000000          0.500000       0.000000   
        13        2.000000    2.000000          0.000000       0.000000   
        20        2.500000    1.750000          0.000000       1.250000   
        25        3.000000    1.666667          0.333333       0.000000   
...                    ...         ...               ...            ...   
73      96        2.875000    3.468750          1.375000       1.031250   
        145       2.301887    4.150943          0.792453       0.566038   
74      84        2.666667   32.666667          2.000000       1.166667   
        96        2.875000    3.468750          1.375000       1.031250   
        145       2.301887    4.150943          0.792453       0.566038   

                  no_blocking  no_blockedby  no_fixversion_change  \
boardid sprintid                                                    
1       8                 0.0           0.0              1.000000   
        12                0.0           0.0              0.500000   
        13                0.0           0.0              1.000000   
        20                0.0           0.0              1.500000   
        25                0.0           0.0              3.333333   
...                       ...           ...                   ...   
73      96                0.0           0.0              1.125000   
        145               0.0           0.0              0.528302   
74      84                0.0           0.0              1.666667   
        96                0.0           0.0              1.125000   
        145               0.0           0.0              0.528302   

                  no_priority_change  no_des_change  type_Bug  ...  type_Epic  \
boardid sprintid                                               ...              
1       8                   0.000000       0.000000  0.000000  ...        0.0   
        12                  0.000000       0.000000  1.000000  ...        0.0   
        13                  0.000000       0.000000  1.000000  ...        0.0   
        20                  0.000000       0.000000  0.500000  ...        0.0   
        25                  0.000000       0.000000  0.666667  ...        0.0   
...                              ...            ...       ...  ...        ...   
73      96                  0.000000       0.281250  0.312500  ...        0.0   
        145                 0.056604       0.301887  0.396226  ...        0.0   
74      84                  0.333333       0.833333  0.333333  ...        0.0   
        96                  0.000000       0.281250  0.312500  ...        0.0   
        145                 0.056604       0.301887  0.396226  ...        0.0   

                  type_Improvement  type_New Feature  type_Story  \
boardid sprintid                                                   
1       8                 0.666667               0.0         0.0   
        12                0.000000               0.0         0.0   
        13                0.000000               0.0         0.0   
        20                0.250000               0.0         0.0   
        25                0.333333               0.0         0.0   
...                            ...               ...         ...   
73      96                0.218750               0.0         0.0   
        145               0.320755               0.0         0.0   
74      84                0.333333               0.0         0.0   
        96                0.218750               0.0         0.0   
        145               0.320755               0.0         0.0   

                  type_Sub-task  type_Task  type_Technical task  type_Test  \
boardid sprintid                                                             
1       8              0.000000   0.333333       

#### Join Iteration Dataset with Issue Dataset

In [44]:
joined_data = apache_iter_30.join(aggr_apache_issue,on=["boardid","sprintid"],how="inner")
joined_data

boardid  sprintid                       name  vel_diff  planday  \
0          1         8             Q2'14 Sprint 1         0        9   
1          1        12             Q2'14 Sprint 2         3       13   
2          1        13                Q2 Sprint 3        -1       12   
3          1        20                Q2 Sprint 4         0       12   
5          1        25                Q3 Sprint 2         0        9   
..       ...       ...                        ...       ...      ...   
343       73        96     Community Day 1 - 2015        -8        4   
344       73       145     Community Day 2 - 2015       -24        3   
345       74        84  Sharan - Jira Test Sprint        -4        7   
346       74        96     Community Day 1 - 2015        -8        4   
347       74       145     Community Day 2 - 2015       -24        3   

     no_issue_starttime  vel_starttime  no_issue_added  vel_added  \
0                     0              0               3          3   
1                     0              0               0          0   
2                     0              0               1          1   
3                     3              3               1          1   
5                     0              0               2          4   
..                  ...            ...             ...        ...   
343                   3              6              13         26   
344                  16             32              19         38   
345                   4              8               2          4   
346                   3              6              13         26   
347                  16             32              19         38   

     no_issue_removed  ...  type_Epic  type_Improvement  type_New Feature  \
0                   3  ...        0.0          0.666667               0.0   
1                   0  ...        0.0          0.000000               0.0   
2                   0  ...        0.0          0.000000               0.0   
3                   4  ...        0.0          0.250000               0.0   
5                   2  ...        0.0          0.333333               0.0   
..                ...  ...        ...               ...               ...   
343                 0  ...        0.0          0.218750               0.0   
344                 0  ...        0.0          0.320755               0.0   
345                 1  ...        0.0          0.333333               0.0   
346                 0  ...        0.0          0.218750               0.0   
347                 0  ...        0.0          0.320755               0.0   

     type_Story  type_Sub-task  type_Task  type_Technical task  type_Test  \
0           0.0       0.000000   0.333333                  0.0        0.0   
1           0.0       0.000000   0.000000                  0.0        0.0   
2           0.0       0.000000   0.000000                  0.0        0.0   
3           0.0       0.000000   0.250000                  0.0        0.0   
5           0.0       0.000000   0.000000                  0.0        0.0   
..          ...            ...        ...                  ...        ...   
343         0.0       0.437500   0.031250                  0.0        0.0   
344         0.0       0.283019   0.000000                  0.0        0.0   
345         0.0       0.000000   0.166667                  0.0        0.0   
346         0.0       0.437500   0.031250                  0.0        0.0   
347         0.0       0.283019   0.000000                  0.0        0.0   

     type_Umbrella  type_Wish  
0              0.0   0.000000  
1              0.0   0.000000  
2              0.0   0.000000  
3              0.0   0.000000  
5              0.0   0.000000  
..             ...        ...  
343            0.0   0.000000  
344            0.0   0.000000  
345            0.0   0.166667  
346            0.0   0.000000  
347            0.0   0.000000  

[347 rows x 40 columns]

In [45]:
joined_data.shape

(347, 40)

### Check Number of Project Iterations

In [46]:
#display counts of under/achieved/over iterations
def count_iter(joined_data):
    under = 0 #vel_diff < 0
    over = 0 #vel_diff > 0
    achieve = 0 #vel_diff = 0
    for i in range(len(joined_data)):
        if joined_data["vel_diff"].iloc[i] < 0:
            under += 1
        elif joined_data["vel_diff"].iloc[i] == 0:
            achieve += 1
        elif joined_data["vel_diff"].iloc[i] > 0:
            over += 1
    print('underachieving iterations: ',under)
    print('overachieving iterations: ',over)
    print('on target iterations: ',achieve)

count_iter(joined_data)

underachieving iterations:  263
overachieving iterations:  42
on target iterations:  42


## Building Prediction Model

### Feature Selection for Prediction Model

#### TEST FEATURE IMPORTANCE

In [47]:
col_drop = ['name','vel_diff','boardid','sprintid','priority']
X = joined_data.drop(col_drop,axis=1)
Y = joined_data.loc[:,'vel_diff']
rfr = RandomForestRegressor()
rfr.fit(X, Y)
features = X.columns
importances = rfr.feature_importances_
indices = np.argsort(importances)  # sort result
plt.figure(figsize=(16,12))
plt.title('Feature Importance to Vel_Diff')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [48]:
joined_data.columns

Index(['boardid', 'sprintid', 'name', 'vel_diff', 'planday',
       'no_issue_starttime', 'vel_starttime', 'no_issue_added', 'vel_added',
       'no_issue_removed', 'vel_removed', 'no_issuetodo', 'vel_todo',
       'no_issueinprogress', 'vel_inprogress', 'no_issuedone', 'vel_done',
       'no_admin', 'no_teammember', 'priority', 'no_comment',
       'no_affectversion', 'no_fixversion', 'no_blocking', 'no_blockedby',
       'no_fixversion_change', 'no_priority_change', 'no_des_change',
       'type_Bug', 'type_Documentation', 'type_Epic', 'type_Improvement',
       'type_New Feature', 'type_Story', 'type_Sub-task', 'type_Task',
       'type_Technical task', 'type_Test', 'type_Umbrella', 'type_Wish'],
      dtype='object')

#### TEST FEATURE CORRELATION

In [49]:
##Correlation analysis
#Correlation plot
df_corr = X.loc[:,['no_teammember', 'planday','no_comment','vel_starttime', 'vel_todo', 'vel_added', 
       'vel_inprogress', 'vel_removed','no_issue_starttime',  'no_issuetodo', 'no_issue_added', 
       'no_issueinprogress','no_issue_removed','no_priority_change','no_des_change']]
#Set the width and height of the plot
f, ax = plt.subplots(figsize=(15, 15))

#Generate correlation matrix
corr = df_corr.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, annot=True, cmap=cmap, vmax=1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

<Axes: >

### Determining the Best Prediction Model

#### Define Accuracy Function

In [50]:
def accuracy(y_actual, y_pred):
    acc = {} # store accuracy
    r2 = metrics.r2_score(y_actual, y_pred)
    mae = metrics.mean_absolute_error(y_actual,y_pred)
    rmse = metrics.mean_squared_error(y_actual, y_pred, squared=False)
    acc['r2'] = r2
    acc['mae'] = mae
    acc['rmse'] = rmse
    print('r2 score:',r2)
    print("MAE:",mae)
    print("RMSE:",rmse)

### StandarScale Data

In [51]:
#final_feature = X.loc[:,['planday', 'vel_todo', 'no_teammember', 'vel_starttime', 'no_issue_starttime']]
X_scaled = StandardScaler().fit_transform(df_corr)

#### Data Split

In [52]:
X_train,X_test,y_train,y_test=train_test_split(X_scaled,Y,test_size=0.2,random_state=10)
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.2,random_state=10)
print('Number of train data:',len(X_train))
print('Number of validation data:',len(X_val))
print('Number of test data:',len(X_test))

Number of train data: 221
Number of validation data: 56
Number of test data: 70


### Prediction Model Selection

#### KNN

In [53]:
params = {'n_neighbors':[2,3,4,5,6,7,8,9]}
knn = neighbors.KNeighborsRegressor()
model = GridSearchCV(knn, params, cv=5)
model.fit(X_train,y_train)
model.best_params_

{'n_neighbors': 3}

In [54]:
knn = neighbors.KNeighborsRegressor(n_neighbors=3)
knn.fit(X_train,y_train)
y_pred = knn.predict(X_val)
accuracy(y_val, y_pred)

r2 score: 0.4968220283014715
MAE: 6.238095238095238
RMSE: 9.54500617376138


#### Decision Tree Regressor

In [55]:
dt = DecisionTreeRegressor(max_depth=200, random_state=None)
dt.fit(X_train,y_train)
y_pred = dt.predict(X_val)
accuracy(y_val, y_pred)

r2 score: 0.4417653637188169
MAE: 6.3125
RMSE: 10.053650722854005


#### Random Forest Regressor

In [56]:
#Create Prediction Model
#col_drop = ['boardid','sprintid','name','vel_diff']
#X = joined_data.loc[:,['vel_todo','no_teammember','vel_starttime', 'no_issue_starttime']]
#apache_iter_30.head()
#Y = joined_data.loc[:,"vel_diff"]
#X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=10)

rfr = RandomForestRegressor(n_estimators=100, max_features='log2', max_depth=1000)
rfr.fit(X_train, y_train)
y_pred_rfr = rfr.predict(X_val)

accuracy(y_val, y_pred_rfr)

r2 score: 0.6964020558002557
MAE: 4.801148809523809
RMSE: 7.414208872386387


In [57]:
from sklearn.model_selection import cross_validate
cv_results = cross_validate(rfr, X_train, y_train, cv=10, scoring=('r2', 'neg_mean_squared_error'),
                            return_train_score=True)
sorted(cv_results.keys())
cv_results['test_r2']
print("cross validate acc: ", np.mean(cv_results['test_r2']))

cross validate acc:  0.6673577861864733


## JADI GINI 

## Evaluate Prediction Model

In [58]:
y_test_pred = rfr.predict(X_test)
accuracy(y_test, y_test_pred)

r2 score: 0.7785191894385823
MAE: 5.161773809523809
RMSE: 7.887737349559437


In [59]:
y_test_pred = knn.predict(X_test)
accuracy(y_test, y_test_pred)

r2 score: 0.7948907086886721
MAE: 5.261904761904763
RMSE: 7.590616596658029


In [60]:
y_test_pred = dt.predict(X_test)
accuracy(y_test, y_test_pred)

r2 score: 0.5944572469748572
MAE: 5.0285714285714285
RMSE: 10.673398173563497


In [61]:
joined_data.shape

(347, 40)